In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-academia.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-wiki.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-cleaned-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-iium.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-parliament.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-watpadd.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-cleaned-common-crawl.txt

files = ['filtered-dumping-academia.txt',
        'filtered-dumping-wiki.txt',
        'dumping-cleaned-news.txt',
        'dumping-iium.txt',
        'dumping-parliament.txt',
        'dumping-watpadd.txt',
        'filtered-dumping-cleaned-common-crawl.txt']

In [3]:
import os
import string

vocabs = list(string.ascii_lowercase + string.digits) + [' ']
directory = '/home/husein/pure-text'

In [3]:
import unicodedata
import re
import itertools

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [ ]:
texts = []

for f in files:
    print(f)
    with open(os.path.join(directory, f)) as fopen:
        text = list(filter(None, fopen.read().split('\n')))
        texts.extend(text)

filtered-dumping-academia.txt
filtered-dumping-wiki.txt
dumping-cleaned-news.txt
dumping-iium.txt
dumping-parliament.txt
dumping-watpadd.txt
filtered-dumping-cleaned-common-crawl.txt


In [ ]:
import json

with open('bahasa-asr-train-combined.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

with open('bahasa-asr-test.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

In [ ]:
import mp
from tqdm import tqdm

def loop(texts):
    texts, _ = texts
    cleaned_texts = []
    for i in tqdm(range(len(texts))):
        t = preprocessing_text(texts[i])
        if len(t):
            cleaned_texts.append(t)
    return cleaned_texts

cleaned_texts = mp.multiprocessing(texts, loop, cores = 16)

In [ ]:
len(texts), len(cleaned_texts)

In [ ]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(cleaned_texts))

In [ ]:
!./kenlm/build/bin/lmplz --text text.txt --arpa out.arpa -o 3 --prune 0 1 1

In [ ]:
!./kenlm/build/bin/build_binary -q 8 -b 7 -a 256 trie out.arpa out.trie.klm

In [1]:
!rm text.txt out.arpa

In [4]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [5]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [ ]:
outPutname = 'language-model/dump-combined/model.trie.klm'
b2_bucket.upload_local_file(
    local_file='out.trie.klm',
    file_name=outPutname,
    file_infos=file_info,
)